<a href="https://colab.research.google.com/github/elricwan/NMP-GPT2-Tutorial/blob/main/story_hf_trainer_model_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction:


## Some infomation about this task:

About the task which we will show here is story generation.

1. Story generation: We will use the GPT-2 to train a model which can generate some stories.
2. Dataset: In huggingface "KATANABRAVE/stories"
3. [GPT model](https://huggingface.co/docs/transformers/v4.32.0/en/model_doc/gpt2#transformers.GPT2Model), we will use the model via huggingface.


Before run this notebook, please ensure that these packages you have already installed.

Python version: 3.8

Packages:
numpy pandas torch torchvision torch-optimizer tqdm accelerate transformers matplotlib datasets huggingface-hub sentencepiece argparse tensorboard

**If not**, please run these codes to install all the package whcih we need. And if you have more packages whcih you want to usem. Please add them in the requirements.txt. When you upload the project, please upload the requirements.txt which you modified.

In [ ]:
!git clone https://github.com/protagolabs/NMP-GPT2-Tutorial.git
!pip install -r NMP-GPT2-Tutorial/requirements.txt
!pip install git+https://github.com/protagolabs/NetMind-Mixin-Runtime@py310

Cloning into 'NMP-GPT2-Tutorial'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 17 (delta 5), reused 13 (delta 4), pack-reused 0
Receiving objects: 100% (17/17), 7.13 KiB | 7.13 MiB/s, done.
Resolving deltas: 100% (5/5), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/protagolabs/NetMind-Mixin-Runtime (to revision py310) to /tmp/pip-req-build-freq_9bp
  Running command git clone --filter=blob:none --quiet https://github.com/protagolabs/NetMind-Mixin-Runtime /tmp/pip-req-build-freq_9bp
  Running command git checkout -b py310 --track origin/py310
  Switched to a new branch 'py310'
  Branch 'py310' set up to track remote branch 'py310' from 'origin'.
  Resolved https://github.com/protagolabs/NetMind-Mixin-Runtime to commit ec06b593cc779ca80da4e9f15851c08826252dd2
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## Not-Netmid-Part


### Step 1: Load the model and tokenizer

In [ ]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained("gpt2",device_map='balanced')
model.train()
"""
Model Parallel Command
"""
model.parallelize()

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Aug 29 01:37:20.537 [INFO] [accelerate.utils.modeling.get_balanced_memory:795] We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

### Step 2: Prepare the dataset.


In [ ]:
from datasets import load_dataset
from transformers import DataCollator


# Import the dataset, which is a demo for some D&D stories.
dataset = load_dataset("KATANABRAVE/stories")




Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

### Step 3: Define the TrainingArguments


In [ ]:
# HuggingFace Trainer
import transformers
from transformers import TrainingArguments

# Here we want to close the wandb, if we use the huggingface's tranier.
import os
os.system("wandb offline")

training_args = TrainingArguments(
    f"saved_model",
    seed = 32,
    #evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    gradient_accumulation_steps=1,
    max_grad_norm=1,
    logging_steps=100,
    save_total_limit=3,
    warmup_steps=200,
    num_train_epochs=1000,
    max_steps=1000,
    save_steps=500,
    fp16=False,
    report_to="none",
)


### Step 4: Define the optimizer and scheduler.

In [ ]:
from transformers import get_linear_schedule_with_warmup, AdamW

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': training_args.weight_decay},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
optimizer = AdamW(optimizer_grouped_parameters, lr=training_args.learning_rate)

schedule_total = training_args.max_steps

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=training_args.warmup_steps, num_training_steps=schedule_total
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Netmid-Part

### Step 5: Initialize the Netmind nmp

In [ ]:
import os
os.environ["PLATFORM"] = os.getenv("PLATFORM", "pytorch")
os.environ["MASTER_ADDR"] = os.getenv("MASTER_ADDR", "127.0.0.1")
os.environ["MASTER_PORT"] = os.getenv("MASTER_PORT", "8000")
os.environ["WORLD_SIZE"] = os.getenv("WORLD_SIZE", "1")
os.environ["RANK"] = os.getenv("RANK", "0")


from NetmindMixins.Netmind import nmp, NetmindOptimizer, NetmindDistributedModel
"""
We turn off the ddp function for model parallel.
"""
nmp.init(use_ddp=False)


### Step 6: Define the NetmindTrainerCallback

We will use it in the trainer initialize

In [ ]:
import transformers
from NetmindMixins.Netmind import NetmindTrainerCallback

class CustomTrainerCallback(NetmindTrainerCallback):
    def __init__(self):
        super().__init__()

    '''
    Add custom training metrics
    '''

    def on_step_end(self, args: transformers.TrainingArguments, state: transformers.TrainerState,
                    control: transformers.TrainerControl, **kwargs):
        kwargs["custom_metrics"] = {}
        return super().on_step_end(args, state, control, **kwargs)

    '''
    Add custom evaluation metrics
    '''

    def on_evaluate(self, args: transformers.TrainingArguments, state: transformers.TrainerState,
                    control: transformers.TrainerControl, **kwargs):
        kwargs["custom_metrics"] = {}
        return super().on_evaluate(args, state, control, **kwargs)

### Setp 7: Start Training

In [ ]:
from transformers import Trainer

nmp.init_train_bar(max_steps=training_args.max_steps)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    optimizers=(optimizer, scheduler),
    callbacks=[CustomTrainerCallback]
)
trainer.remove_callback(transformers.trainer_callback.PrinterCallback)
trainer.remove_callback(transformers.trainer_callback.ProgressCallback)

trainer.train()

nmp.finish_training() # Finish the training. It should be placed at the end of file

Step,Training Loss
100,3.555800
200,3.151300
300,3.103100
400,3.060200
500,3.012200
600,3.024800
700,2.936200
800,2.884000
900,2.947700
1000,2.948400


INFO:NetmindMixins.NetmindModelMixin:progress: 18
Aug 29 01:07:16.412 [INFO] [NetmindMixins.NetmindModelMixin.on_step_end:15] progress: 18
INFO:NetmindMixins.NetmindModelMixin:progress: 37
Aug 29 01:09:16.510 [INFO] [NetmindMixins.NetmindModelMixin.on_step_end:15] progress: 37
INFO:NetmindMixins.NetmindModelMixin:progress: 55
Aug 29 01:11:17.008 [INFO] [NetmindMixins.NetmindModelMixin.on_step_end:15] progress: 55
INFO:NetmindMixins.NetmindModelMixin:progress: 75
Aug 29 01:13:17.457 [INFO] [NetmindMixins.NetmindModelMixin.on_step_end:15] progress: 75
INFO:NetmindMixins.NetmindModelMixin:progress: 94
Aug 29 01:15:17.904 [INFO] [NetmindMixins.NetmindModelMixin.on_step_end:15] progress: 94
